# Competition Objective is to detect fraud in transactions; 

## Data


In this competition you are predicting the probability that an online transaction is fraudulent, as denoted by the binary target ```isFraud```.

The data is broken into two files **identity** and **transaction**, which are joined by ```TransactionID```. 

> Note: Not all transactions have corresponding identity information.

**Transaction variables**

- TransactionDT: timedelta from a given reference datetime (not an actual timestamp)
- TransactionAMT: transaction payment amount in USD
- ProductCD: product code, the product for each transaction
- card1 - card6: payment card information, such as card type, card category, issue bank, country, etc.
- addr: address
- dist: distance
- P_ and (R__) emaildomain: purchaser and recipient email domain
- C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
- D1-D15: timedelta, such as days between previous transaction, etc.
- M1-M9: match, such as names on card and address, etc.
- Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.

**Categorical Features - Transaction**

- ProductCD
- emaildomain
- card1 - card6
- addr1, addr2
- P_emaildomain
- R_emaildomain
- M1 - M9

**Categorical Features - Identity**

- DeviceType
- DeviceInfo
- id_12 - id_38

**The TransactionDT feature is a timedelta from a given reference datetime (not an actual timestamp).**


# 1. Importation and memory reduction
## 1.1. Importing necessary libraries

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import datetime


# Standard plotly imports
#import plotly.plotly as py
import plotly.graph_objs as go
import ipywidgets
import plotly.express as px
import plotly.tools as tls
from plotly.subplots import make_subplots

from plotly.offline import iplot, init_notebook_mode
#import cufflinks
#import cufflinks as cf
import plotly.figure_factory as ff


# Using plotly + cufflinks in offline mode
init_notebook_mode(connected=True)
#cufflinks.go_offline(connected=True)

# Preprocessing, modelling and evaluating
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold, GridSearchCV
from xgboost import XGBClassifier
import xgboost as xgb
import catboost as cb

## Hyperopt modules
from functools import partial

import os
import gc
import time

## 1.2. Importing train datasets

In [3]:
gc.collect()
df_train = pd.read_pickle("df_train_v2.pkl")
df_test = pd.read_pickle("df_test_v2.pkl")
sample_submission = pd.read_csv('sample_submission.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'df_train_v2.pkl'

In [6]:
print(df_train.shape)
print(df_test.shape)

(590540, 430)
(506691, 429)


## 1.3. Memory reduction

In [0]:
def resumetable(df):
    n = df.shape[0]
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values  
    summary['Missing %'] = round(summary['Missing'] / n * 100,2)
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values


    return summary


To see the output of the Resume Table, click to see the output 

# 2. Preprocessing

In [8]:
df_train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,D10,...,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_19,id_20,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,_merge,dist1_na,dist2_na,_Weekdays,_Hours,_Days
0,2987000,0,86400,4.226562,W,13926,-inf,150,discover,142,credit,315,87,19.0,0.0,gmail.com,gmail.com,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,97.0,13.0,26.0,10.0,0.0,37.875,0.666504,13.0,...,0.0,0.0,0.0,0.0,70787.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,NotFound,52.0,-480.0,New,NotFound,166.0,542.0,144.0,Miss,New,NotFound,Android,Samsung,32.0,2220x1080,match_status:2,T,F,T,T,mobile,Others,both,False,True,5,0,2
1,2987001,0,86401,3.367188,W,2755,404,150,mastercard,102,credit,325,87,8.0,0.0,gmail.com,Maybe_P,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,97.0,8.0,0.0,10.0,0.0,37.875,0.666504,0.0,...,0.0,0.0,0.0,-5.0,98945.0,0.0,0.0,0.0,-5.0,0.0,0.0,100.0,NotFound,49.0,-300.0,New,NotFound,166.0,621.0,500.0,Miss,New,NotFound,iOS,Safari,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device,both,True,True,5,0,2
2,2987002,0,86469,4.078125,W,4663,490,150,visa,166,debit,330,87,287.0,0.0,outlook.com,Maybe_P,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,97.0,8.0,0.0,10.0,0.0,37.875,0.666504,0.0,...,0.0,0.0,0.0,-5.0,191631.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,NotFound,52.0,-300.0,Found,Found,121.0,410.0,142.0,Miss,Found,Found,Miss,Chrome,24.0,1920x1080,match_status:2,F,F,T,T,desktop,Windows,both,False,True,5,0,2
3,2987003,0,86499,3.912109,W,18132,567,150,mastercard,117,debit,476,87,8.0,0.0,yahoo.com,Maybe_P,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,0.0,37.875,0.666504,84.0,...,0.0,0.0,0.0,-5.0,221832.0,0.0,0.0,0.0,-6.0,0.0,0.0,100.0,NotFound,52.0,-300.0,New,NotFound,225.0,176.0,507.0,Miss,New,NotFound,Miss,Chrome,24.0,1920x1080,match_status:2,F,F,T,T,desktop,Windows,both,True,True,5,0,2
4,2987004,0,86506,3.912109,H,4497,514,150,mastercard,102,credit,420,87,8.0,0.0,gmail.com,Maybe_P,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,97.0,8.0,26.0,10.0,0.0,37.875,0.666504,15.0,...,0.0,0.0,0.0,0.0,7460.0,0.0,0.0,1.0,0.0,0.0,0.0,100.0,NotFound,52.0,-300.0,Found,Found,166.0,529.0,575.0,Miss,Found,Found,Mac,Chrome,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS,both,True,True,5,0,2


In [0]:
categorical_features = []
for col in df_train.columns.drop('isFraud') :
    if (df_train[col].dtype == 'object' 
        or df_test[col].dtype=='object' 
        or df_train[col].dtype == 'bool') :
        categorical_features.append(col)
categorical_features.extend(['_merge', '_Weekdays',	'_Hours',	'_Days'])

In [10]:
categorical_resume = resumetable(df_train[categorical_features])
categorical_resume

Dataset Shape: (590540, 42)


,Name,dtypes,Missing,Missing %,Uniques,First Value,Second Value,Third Value
0,ProductCD,object,0,0.0,5,W,W,W
1,card1,object,0,0.0,13553,13926,2755,4663
2,card2,object,0,0.0,501,-inf,404,490
3,card3,object,0,0.0,115,150,150,150
4,card4,object,0,0.0,5,discover,mastercard,visa
5,card5,object,0,0.0,120,142,102,166
6,card6,object,0,0.0,5,credit,credit,debit
7,addr1,object,0,0.0,333,315,325,330
8,addr2,object,0,0.0,75,87,87,87
9,P_emaildomain,object,0,0.0,31,gmail.com,gmail.com,outlook.com


In [0]:
# Label Encoding
categorical_features_v2 = categorical_features.copy()
for f in categorical_features:
    if float(categorical_resume.loc[categorical_resume['Name']==f, 'Uniques']) > 8 :
        categorical_features_v2.remove(f)
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df_train[f].values) + list(df_test[f].values))
        df_train[f] = lbl.transform(list(df_train[f].values))
        df_test[f] = lbl.transform(list(df_test[f].values))  

In [12]:
categorical_resume_v2 = resumetable(df_train[categorical_features_v2])
categorical_resume_v2

Dataset Shape: (590540, 30)


,Name,dtypes,Missing,Missing %,Uniques,First Value,Second Value,Third Value
0,ProductCD,object,0,0.0,5,W,W,W
1,card4,object,0,0.0,5,discover,mastercard,visa
2,card6,object,0,0.0,5,credit,credit,debit
3,M1,object,0,0.0,3,T,Miss,T
4,M2,object,0,0.0,3,T,Miss,T
5,M3,object,0,0.0,3,T,Miss,T
6,M4,object,0,0.0,4,M2,M0,M0
7,M5,object,0,0.0,3,F,T,F
8,M6,object,0,0.0,3,T,T,F
9,M7,object,0,0.0,3,Miss,Miss,F


In [0]:
Y_train = df_train['isFraud']
df_train.drop('isFraud', axis = 1, inplace = True)

In [0]:
categorical_index = [df_train.columns.get_loc(key) for key in categorical_features_v2]


# 3. Modeling

In [16]:
train = cb.Pool(df_train, Y_train, cat_features=categorical_index)
test = cb.Pool(df_test, cat_features=categorical_index)
del df_train, Y_train, df_test
gc.collect()


65

In [18]:
params = {'depth': [15],
          'learning_rate' : [0.03],
         'l2_leaf_reg': [3],
         'iterations': [1000]}

cb_model = cb.CatBoostClassifier(eval_metric="AUC", 
                                 one_hot_max_size= 8, 
                                 task_type="GPU",
                                 devices='0:1',
                                 verbose = 200,
                                 depth = 15,
                                 l2_leaf_reg = 3,
                                 iterations = 1000)
cb_model.fit(train)

0:	learn: 0.8108965	total: 1.21s	remaining: 20m 10s
200:	learn: 0.9665239	total: 3m 50s	remaining: 15m 15s
400:	learn: 0.9848535	total: 7m 36s	remaining: 11m 22s
600:	learn: 0.9914796	total: 11m 17s	remaining: 7m 29s
800:	learn: 0.9953011	total: 14m 57s	remaining: 3m 43s
999:	learn: 0.9973242	total: 18m 37s	remaining: 0us


In [19]:
cb_model.get_all_params()

{'add_ridge_penalty_to_loss_function': False,
 'bagging_temperature': 1,
 'bayesian_matrix_reg': 0.10000000149011612,
 'best_model_min_trees': 1,
 'boost_from_average': False,
 'boosting_type': 'Plain',
 'bootstrap_type': 'Bayesian',
 'border_count': 128,
 'class_names': [0, 1],
 'classes_count': 0,
 'combinations_ctr': ['Borders:CtrBorderCount=15:CtrBorderType=Uniform:TargetBorderCount=1:TargetBorderType=MinEntropy:Prior=0/1:Prior=0.5/1:Prior=1/1',
  'FeatureFreq:CtrBorderCount=15:CtrBorderType=Median:Prior=0/1'],
 'counter_calc_method': 'SkipTest',
 'ctr_history_unit': 'Sample',
 'ctr_target_border_count': 1,
 'data_partition': 'FeatureParallel',
 'depth': 15,
 'devices': '0:1',
 'eval_metric': 'AUC',
 'feature_border_type': 'GreedyLogSum',
 'fold_permutation_block': 64,
 'fold_size_loss_normalization': False,
 'gpu_cat_features_storage': 'GpuRam',
 'gpu_ram_part': 0.95,
 'grow_policy': 'SymmetricTree',
 'has_time': False,
 'iterations': 1000,
 'l2_leaf_reg': 3,
 'leaf_estimation_bac

In [0]:
sample_submission['isFraud'] = cb_model.predict_proba(test)[:,1]

In [22]:
from google.colab import drive
drive.mount('/content/gdrive')



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
cb_model.save_model('/content/gdrive/My Drive/model_cat')

In [0]:
sample_submission.to_csv('/content/gdrive/My Drive/submission.csv', index=False)

In [0]:
grid_search_result = cb_model.grid_search(params,
                                          cv = 5,
                                          X=train,
                                          stratified = True,
                                          plot=True)